In [2]:
!ls ../Data/data_set_phase1/

profiles.csv	test_queries.csv  train_plans.csv
test_plans.csv	train_clicks.csv  train_queries.csv


In [4]:
import pandas as pd
import numpy as np
main_path = r'../Data/data_set_phase1/'

# Train data

## Train queries

In [8]:
train_queries_df_copy = pd.read_csv(main_path+'train_queries.csv')
train_queries_df = train_queries_df_copy.copy()

In [9]:
train_queries_df.head()
# sid: session ID
# pid: profile ID
# req_time: request time
# o: original point
# d: destination point

,sid,pid,req_time,o,d
0,3000821,NaN,2018-11-02 17:54:30,"116.29,39.97","116.32,39.96"
1,3085857,210736.0,2018-11-16 10:53:10,"116.39,39.84","116.33,39.79"
2,2944522,NaN,2018-10-06 10:33:58,"116.31,39.93","116.27,40.00"
3,559931,202427.0,2018-11-23 14:54:11,"116.27,39.88","116.39,39.90"
4,2819352,172251.0,2018-10-30 11:48:41,"116.34,39.96","116.37,39.86"


In [13]:
train_queries_df.isnull().sum()

sid              0
pid         163979
req_time         0
o                0
d                0
dtype: int64

In [14]:
163979/train_queries_df.shape[0]

0.327958

In [16]:
train_queries_df.nunique()/train_queries_df.shape[0]

sid         1.000000
pid         0.092382
req_time    0.928422
o           0.010920
d           0.009846
dtype: float64

In [18]:
train_queries_df['o'].value_counts()[:10]

116.37,39.86    5993
116.32,39.89    5930
116.42,39.90    5819
116.41,39.91    5616
116.37,39.91    5246
116.40,39.91    5156
116.39,39.90    4366
116.35,39.94    3674
116.41,39.90    3048
116.39,39.91    3048
Name: o, dtype: int64

In [19]:
train_queries_df['d'].value_counts()[:10]

116.32,39.89    16503
116.37,39.86    13124
116.42,39.90    11256
116.39,39.90     7575
116.39,39.91     7571
116.41,39.91     6348
116.37,39.91     5665
116.45,39.93     5357
116.27,40.00     4222
116.40,39.93     3782
Name: d, dtype: int64